In [2]:
import os
import numpy as np
import torch

In [3]:
import json
h5py_path = os.path.join(os.path.dirname(os.getcwd()), 'logs/rsl_rl/Genhexapod2_genhexapod__KneeNum_l1-0_l2-0_l3-0_l4-0_l5-0_l6-0__ScaleJointLimit_l1-0_l2-0_l3-0_l4-0_l5-0_l6-0_1_0__Geo_scale_all_0_8/2024-12-18_10-38-06/h5py_record/obs_actions_00000.h5')

In [4]:
import h5py
max_parallel_envs_per_file = 4096
with h5py.File(h5py_path, "r") as data_file:
            inputs = np.array(data_file["one_policy_observation"][:, :max_parallel_envs_per_file])
            targets = np.array(data_file["actions"][:, :max_parallel_envs_per_file])

In [5]:
inputs.shape

(200, 4096, 268)

In [7]:
class Metadata:
    def __init__(self, nr_dynamic_joint_observations, single_dynamic_joint_observation_length,
                 dynamic_joint_observation_length, dynamic_joint_description_size,
                 trunk_angular_vel_update_obs_idx, goal_velocity_update_obs_idx,
                 projected_gravity_update_obs_idx):
        self.nr_dynamic_joint_observations = nr_dynamic_joint_observations
        self.single_dynamic_joint_observation_length = single_dynamic_joint_observation_length
        self.dynamic_joint_observation_length = dynamic_joint_observation_length
        self.dynamic_joint_description_size = dynamic_joint_description_size
        self.trunk_angular_vel_update_obs_idx = trunk_angular_vel_update_obs_idx
        self.goal_velocity_update_obs_idx = goal_velocity_update_obs_idx
        self.projected_gravity_update_obs_idx = projected_gravity_update_obs_idx

# Example instantiation with sample values
metadata = Metadata(
    nr_dynamic_joint_observations=12,
    single_dynamic_joint_observation_length=21,
    dynamic_joint_observation_length=252,
    dynamic_joint_description_size=18,
    trunk_angular_vel_update_obs_idx=[252, 253, 254],  # Example indices
    goal_velocity_update_obs_idx=[255, 256, 257],
    projected_gravity_update_obs_idx=[258, 259, 260]
)

In [13]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd())+"/scripts/rsl_rl")
from utils import one_policy_observation_to_inputs
inputs_tensor = torch.tensor(inputs)
dynamic_joint_description = []
dynamic_joint_state = []
general_policy_state = []
for i in range(200):
    (
        dynamic_joint_description_i,
        dynamic_joint_state_i,
        general_policy_state_i
    ) = one_policy_observation_to_inputs(inputs_tensor[i], metadata, 'cuda: 0')
    dynamic_joint_description.append(dynamic_joint_description_i)
    dynamic_joint_state.append(dynamic_joint_state_i)
    general_policy_state.append(general_policy_state_i)


dynamic_joint_description = torch.stack((dynamic_joint_description), dim=0)
dynamic_joint_state = torch.stack((dynamic_joint_state), dim=0)
general_policy_state = torch.stack((general_policy_state), dim=0)

In [18]:
dynamic_joint_state.shape

torch.Size([200, 4096, 12, 3])

In [ ]:
targets.shape

(200, 4096, 12)

In [ ]:
dynamic_joint_description.shape

torch.Size([200, 4096, 12, 18])

In [19]:
general_policy_state.shape

torch.Size([200, 4096, 16])

In [15]:
import json
configure_path = os.path.join(os.path.dirname(os.getcwd()), 'exts/berkeley_humanoid/berkeley_humanoid/assets/Robots/GenBot1K-v2/configure/Genhexapod0_307_averageEnv_policy_description.json')

In [16]:
if os.path.isfile(configure_path):
    with open(configure_path, 'r') as f:
        data_file = json.load(f)
        configure = np.array(data_file['Genhexapod2']['dynamic_joint_description'])
else:
    print("[WARNING] File not exists")

In [17]:
configure.shape

(12, 18)